In [1]:
import pandas as pd 
import json 
import re 
import warnings
warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)

In [2]:
# get Bibles 
bible_dir = "/Users/amycweng/DH/SERMONS_APP/db/data/Bibles"
bibles = ['Douay-Rheims','Geneva','KJV','Tyndale','Wycliffe','Vulgate']
bible_verses = {}
for b in bibles: 
    with open(f"{bible_dir}/{b}.json",'r') as f: 
        verses = json.load(f)
    for v in verses: 
        bible_verses[f'{v} ({b})'] = verses[v]
list(bible_verses.keys())[0]


'Genesis 1.1 (Douay-Rheims)'

In [340]:
vname = 'Galatians 5.19'
for key,verse in bible_verses.items(): 
    if re.search(rf"^{vname} \([DKG]",key): 
        print(f"{key}\t\t\t{verse}\tTRUE")

Galatians 5.19 (Douay-Rheims)			Now the works of the flesh are manifest, which are fornication, uncleanness, immodesty, luxury,	TRUE
Galatians 5.19 (Geneva)			Moreouer the workes of the flesh are manifest, which are adulterie, fornication, vncleannes, wantonnes,	TRUE
Galatians 5.19 (KJV)			Now the works of the flesh are manifest, which are these; Adultery, fornication, uncleanness, lasciviousness,	TRUE


In [341]:
for key,verse in bible_verses.items(): 
    if re.search(rf"^{vname} \([V]",key): 
        print(f"{key}\t\t\t{verse}\tTRUE")

Galatians 5.19 (Vulgate)			Manifesta sunt autem opera carnis, quæ sunt fornicatio, immunditia, impudicitia, luxuria,	TRUE


In [338]:
for key,verse in bible_verses.items(): 
    if re.search(rf"^{vname} \([T]",key): 
        print(f"{key}\t\t\t{verse}\tTRUE")

Acts 14.22 (Tyndale)			and strengthed the disciples soules exhortinge them to continue in the faith affyrminge that we must thorowe moche tribulacion entre into the kyngdome of God.	TRUE


In [4]:
with open('../assets/corpora.json','r') as file: 
    corpora = json.load(file)

main_dir = "/Users/amycweng/DH/EEPS"
for era in corpora: 
    qp_file = f"/Users/amycweng/DH/SERMONS_APP/db/data/{era}/paraphrases.csv"
    qp = pd.read_csv(qp_file,header=None)
    qp = qp.sort_values(by=3,ignore_index=True)
    qp[8] = ''
    for idx, qp_label in enumerate(qp[3]): 
        qp[8][idx] = bible_verses[qp_label] 
    qp.to_csv(f"{main_dir}/qp/{era}_qp.csv",index=False)
    print(era)
    break

pre-Elizabethan


In [ ]:
import os 
valid_files = os.listdir(f"{main_dir}/encodings_new")
for era in corpora: 
    qp_file = f"/Users/amycweng/DH/SERMONS_APP/db/data/{era}/paraphrases.csv"
    qp = pd.read_csv(qp_file,header=None)
    tcpIDs = qp[0].unique()
    all_dfs = []
    for tcpID in tcpIDs:
        if f"{tcpID}_encoded.csv" not in valid_files: 
            continue 
        df = pd.read_csv(f"{main_dir}/encodings_new/{tcpID}_encoded.csv")
        df['qp_tag'] = df['qp_tag'].apply(lambda x: x != "True")
        df = df.groupby('sent_idx').agg({
                'token': lambda x: ' '.join(x),  
                'cite_label':lambda x: list(set(x)),
            }).reset_index()
        df['tcpID'] = tcpID
        df['cite_label'] = df['cite_label'].apply(lambda x: [v for v in x if not pd.isna(v)])
        df = df[df['cite_label'].apply(lambda x: x != [])]        
        new_order = ['tcpID','sent_idx','token']
        df = df[new_order]
        all_dfs.append(df)
    all_dfs = pd.concat(all_dfs)
    print(era)
    all_dfs.to_csv(f"{main_dir}/qp/{era}_citations.csv",index=False)